In [1]:
# 데이터 불러오기
import pandas as pd

df = pd.read_csv("../data/CARD_SUBWAY_MONTH_202505.csv")  # 파일명은 상황에 맞게 수정

# 기본 정보 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19139 entries, 20250501 to 20250531
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   사용일자    19139 non-null  object 
 1   노선명     19139 non-null  object 
 2   역명      19139 non-null  int64  
 3   승차총승객수  19139 non-null  int64  
 4   하차총승객수  19139 non-null  int64  
 5   등록일자    0 non-null      float64
dtypes: float64(1), int64(3), object(2)
memory usage: 1.0+ MB
